In [1]:
# Dependencies
import requests
import pymongo
import dns
import datetime
from datetime import datetime
from dateutil import parser
import pytz
from django.utils import timezone
from time import sleep
import random
import json
import config
import requests
from flask_cors import CORS
import os
import csv


# Project defined dependencies
import config
from pprint import pprint
# import cl_parser


In [2]:
# Initialize PyMongo to work with MongoDBs
client = pymongo.MongoClient(config.mongo_conn)
# Define database and collection
db = client.getitdone_db
SR_collection = db.sandiego
summary_collection = db.summary_counts

In [3]:
year = 2021
local = pytz.timezone("America/Los_Angeles")
dt_start = datetime.strptime(str(year) + "-1-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_start_local = local.localize(dt_start, is_dst=None)
dt_start_utc = dt_start_local.astimezone(pytz.utc)

dt_end = datetime.strptime(str(year + 1) +"-1-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_end_local = local.localize(dt_end, is_dst=None)
dt_end_utc = dt_end_local.astimezone(pytz.utc)

filter_requested ={
    'date_requested': {
        '$gte': dt_start_utc, 
        '$lt': dt_end_utc
    }
}
filter_closed ={
    'date_requested': {
        '$gte': dt_start_utc, 
        '$lt': dt_end_utc
    }, 'status':"Closed"
}


total_requested = SR_collection.count_documents(filter=filter_requested)

sum_case_age_days = 0
for SR in SR_collection.find(filter=filter_requested):
    if 'case_age_days' in SR:
        sum_case_age_days += int(SR["case_age_days"])

total_closed = SR_collection.count_documents(filter=filter_closed)

print(dt_start_utc, dt_end_utc)
print(total_requested, total_closed)
print(sum_case_age_days)

2021-01-01 08:00:00+00:00 2022-01-01 08:00:00+00:00
90777 52863
1193134


In [4]:
            update_doc = summary_collection.find_one_and_update(
                {'year' : year},
                {'$set':
                    {
                        "total_requested":int(total_requested),
                        "total_closed":int(total_closed),
                        "percent_closed": float(total_closed/total_requested),
                        "average_case_age_days": float(sum_case_age_days/total_requested)
                        
                    }
                    
                },upsert=True
            )

In [12]:
filter={
    'date_requested': {
        '$gte': dt_start_utc, 
        '$lt': dt_end_utc
    }
}

result = client['getitdone_db']['sandiego'].find(filter=filter)

In [13]:
print(dt_start_utc,dt_end_utc)

2021-01-01 08:00:00+00:00 2022-01-01 08:00:00+00:00


In [14]:
i = 0
date_requested_count = {}  
date_closed_count = {}

for SR in result:
    
    if 'date_requested' in SR:            
        date_requested_month = SR['date_requested'].month     
        if (date_requested_month in date_requested_count):
            date_requested_count[date_requested_month] += 1        
        else:
            date_requested_count[date_requested_month] = 1            
    if 'date_closed' in SR:
        date_closed_month = SR['date_closed'].month     
        if (date_closed_month in date_closed_count):
            date_closed_count[date_closed_month] += 1        
        else:
            date_closed_count[date_closed_month] = 1 

    # i += 1
    # if i > 5:
    #     break

In [15]:
date_requested_count

{1: 30506, 2: 28203, 3: 32872, 4: 143}